# Modelos de spacy

Como vimos en la teoría, en general no vamos a entrenar embeddings desde 0 sino que utilizaremos embeddings ya entrenados y luego, si es necesario, los adaptaremos a nuestros datos (no en esta clase).

En este notebook, queremos acceder a embeddings de palabras. El modelo de spacy con el que venimos trabajando hasta ahora es: [**en_core_web_sm**](https://spacy.io/models/en#en_core_web_sm) (35MB). El mismo nos provee vocabulario, sintaxis, entidades y todo lo que estuvimos viendo hasta ahora, pero no embeddings.

Para utilizar embeddings (en inglés) en spacy tenemos las siguientes opciones: 
> [**en_core_web_md**](https://spacy.io/models/en#en_core_web_md) (116MB) Vectors: 685k keys, 20k unique vectors (300 dimensions)

> [**en_core_web_lg**](https://spacy.io/models/en#en_core_web_lg) (812MB) Vectors: 685k keys, 685k unique vectors (300 dimensions)

> [**en_vectors_web_lg**](https://spacy.io/models/en#en_vectors_web_lg) (631MB) Vectors: 1.1m keys, 1.1m unique vectors (300 dimensions)

En nuestro caso, con  **en_core_web_md** será suficiente. Pueden probar luego los otros modelos.

Por defecto en colab no tenemos los modelos más pesados, por lo que **la siguiente celda les dará un error:**

In [12]:
import numpy as np

In [ ]:
!python -m spacy download en_core_web_md

In [10]:
import spacy
nlp = spacy.load('en_core_web_md')

Por lo tanto debemos descargarlo:

In [ ]:
# !python -m spacy download en_core_web_lg

Luego de instalarlo deberán resetear su runtime de colab.

Para esto van a runtime -> restart runtime.

Ahora si deberían poder cargar el modelo:

In [5]:
# import spacy
# nlp = spacy.load('en_core_web_md')

# Embeddings en spacy

En spacy vimos que los vectores que se utilizan tienen 300 dimensiones.

Podemos acceder a los mismos de la siguiente manera:

In [ ]:
nlp('lion').vector

Si le pedimos el embedding de un documento, lo calculara como el promedio de los vectores de todas las palabras. Con esto, podemos computar similaridad entre documentos.

In [ ]:
doc = nlp(u'The quick brown fox jumped over the lazy dogs.')

doc.vector

# Identificando vectores similares

Spacy nos provee el método .similarity() para evaluar la similitud entre palabras.

El método similarity es de los tokens.

In [5]:
tokens = nlp('lion cat pet')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

lion lion 1.0
lion cat 0.3854507803916931
lion pet 0.20031584799289703
cat lion 0.3854507803916931
cat cat 1.0
cat pet 0.732966423034668
pet lion 0.20031584799289703
pet cat 0.732966423034668
pet pet 1.0


<font color=red>El orden NO importa. `token1.similarity(token2)` retorna lo mismo que `token2.similarity(token1)`.</font>

Como es de esperar, vemos la relación más fuerte entre "cat" y "pet" y la más débil es "lion" con "pet".

# Palabras opuestas

Como palabras opuestas pueden aparecer muchas veces en los mismos contextos, seguramente vamos a encontrar casos con palabras opuestas y distancias pequeñas.

In [6]:
tokens = nlp('like love hate')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

like like 1.0
like love 0.5212638974189758
like hate 0.5065141320228577
love like 0.5212638974189758
love love 1.0
love hate 0.5708349943161011
hate like 0.5065141320228577
hate love 0.5708349943161011
hate hate 1.0


# Operaciones con vectores

También podemos calcular nuevos vectores haciendo sumas y restas. Existe un ejemplo famoso sobre Word2vec que dice que:

<pre>"king" - "man" + "woman" = "queen"</pre>

Lo podemos probar:

In [21]:
from scipy import spatial

#Función para calcular distancia coseno
def cosine_similarity(x, y):
  return 1 - spatial.distance.cosine(x, y) 

king = nlp.vocab['king'].vector
man = nlp.vocab['man'].vector
woman = nlp.vocab['woman'].vector

new_vector = king - man + woman
computed_similarities = []

# Comparamos con todo el vocabulario
for word in nlp.vocab:
    # Ignoramos palabras sin embedding en el modelo
    if word.has_vector:
      #Ignoramos palabras en mayúsculas
      if word.is_lower:
            # Nos quedamos con palabras
            if word.is_alpha:
                if len(word.text)>4:
              #Calculamos distancia coseno
                  similarity = cosine_similarity(new_vector, word.vector)
                  #similarity = norm(new_vector-word.vector)
              # Se almacenan los resultados en la lista de tuplas
              # Cada tupla tiene como primer valor el token y segundo valor la similaridad
                  computed_similarities.append((word, similarity))

# Ordenamos de mayor a menor
computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])

print([f"{w[0].text}:{w[1]}" for w in computed_similarities[:10]])

['queen:0.6178014278411865', 'havin:0.3667823076248169', 'where:0.3385923206806183', 'woman:0.30994713306427', 'somethin:0.30953145027160645', 'there:0.30542072653770447', 'should:0.29837310314178467', 'these:0.29441288113594055', 'would:0.2928425371646881', 'nothin:0.2927696406841278']


In [23]:
king = nlp.vocab['husband '].vector
man = nlp.vocab['man'].vector
woman = nlp.vocab['woman'].vector
nlp.vocab['wife'].vector

new_vector = king - man + woman
computed_similarities = []

# Comparamos con todo el vocabulario
for word in nlp.vocab:
    # Ignoramos palabras sin embedding en el modelo
    if word.has_vector:
      #Ignoramos palabras en mayúsculas
      if word.is_lower:
            # Nos quedamos con palabras
            if word.is_alpha:
                if len(word.text)>3:
              # Calcular la similitud con cada palabra
                    similarity =np.dot(new_vector, word.vector) / (np.linalg.norm(new_vector) * np.linalg.norm(word.vector))
                    computed_similarities.append((word, similarity))

# Ordenar las similitudes por mayor similitud
sorted_similarities = sorted(computed_similarities, key=lambda x: x[1], reverse=True)

print([f"{w[0].text}:{w[1]}" for w in sorted_similarities[:10]])

['wife:0.04696419835090637', 'queen:0.03473014757037163', 'these:0.000905352586414665', 'space:-0.011037594638764858', 'were:-0.014306643046438694', 'should:-0.023915868252515793', 'does:-0.029595397412776947', 'where:-0.0332501120865345', 'need:-0.03333720564842224', 'would:-0.04728491231799126']


En este caso, con estos embeddings no nos dió que la palabra más similar es "queen", pero sale en el 2do lugar!

In [42]:
queen = nlp.vocab['queen'].vector

In [51]:
dog = nlp.vocab['dog']

In [53]:
dog.text

'dog'

In [43]:
from numpy.linalg import norm

In [46]:
norm(queen)

6.82974

In [47]:
norm(new_vector)

7.9464583

In [48]:
norm(queen-new_vector)

4.9243603